In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.insert(0, os.path.abspath('../CaseLawAnalytics/'))
sys.path.insert(0, os.path.abspath('..'))

In [2]:
import zipfile
from lxml import etree
from caselawres import dbutils

In [3]:
conn = dbutils.get_connection()  

In [71]:
conn.execute(''' DROP TABLE IF EXISTS uitspraken_meta''')
conn.execute(''' CREATE TABLE uitspraken_meta
            (id text PRIMARY KEY, 
            title text,
            summary text,
            updated text
            )
        ''')

## Retrieve all HR meta data

In [61]:
def get_entries_from_link(baselink, fr=0, \
                          maximum=1000):
    link = baselink+'&max='+str(maximum)+'&from='+str(fr)
    xml_element = etree.ElementTree().parse(link)
    entries = list(xml_element.iter('{*}entry'))
    return entries

In [62]:

def get_first_content(el, tag):
    return list(el.iter('{*}'+tag))[0].text

def insert_into_uitspraken_meta(entry, curs, table='uitspraken_meta'):
    id0 = get_first_content(entry, 'id')
    title = get_first_content(entry, 'title')
    summary = get_first_content(entry, 'summary')
    updated = get_first_content(entry, 'updated')
    
    query = ''' INSERT OR REPLACE INTO uitspraken_meta
        VALUES (?, ?, ?, ?)
    '''
    curs.execute(query, (id0, title, summary, updated))

In [72]:
baselink = 'http://data.rechtspraak.nl/uitspraken/zoeken?' + \
            'creator=http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden' + \
            '&type=Uitspraak' 
baselink

'http://data.rechtspraak.nl/uitspraken/zoeken?creator=http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden&type=Uitspraak'

In [73]:
size = 1000
end = 98000
for start in range(0, end, size):
    entries = get_entries_from_link(baselink, start, size)
    print(start, len(entries))
    for entry in entries:
        insert_into_uitspraken_meta(entry, conn)

0 1000
1000 1000
2000 1000
3000 1000
4000 1000
5000 1000
6000 1000
7000 1000
8000 1000
9000 1000
10000 1000
11000 1000
12000 1000
13000 1000
14000 1000
15000 1000
16000 1000
17000 1000
18000 1000
19000 1000
20000 1000
21000 1000
22000 1000
23000 1000
24000 1000
25000 1000
26000 1000
27000 1000
28000 1000
29000 1000
30000 1000
31000 1000
32000 1000
33000 1000
34000 1000
35000 1000
36000 1000
37000 1000
38000 1000
39000 1000
40000 1000
41000 1000
42000 1000
43000 1000
44000 1000
45000 1000
46000 1000
47000 1000
48000 1000
49000 1000
50000 1000
51000 1000
52000 1000
53000 1000
54000 1000
55000 1000
56000 1000
57000 1000
58000 1000
59000 1000
60000 1000
61000 1000
62000 1000
63000 1000
64000 1000
65000 1000
66000 1000
67000 1000
68000 1000
69000 1000
70000 1000
71000 1000
72000 1000
73000 1000
74000 1000
75000 1000
76000 1000
77000 1000
78000 1000
79000 1000
80000 1000
81000 1000
82000 1000
83000 1000
84000 1000
85000 1000
86000 1000
87000 1000
88000 1000
89000 1000
90000 1000
91000 1000
9

In [75]:
conn.commit()

## Download RDFs from LiDO

In [86]:
from caselawnet.lido_parser import LinkExtractorXMLParser, LinkExtractorRDFParser

In [99]:
filename = 'settings.cfg'
with open(filename) as f:
    exec(compile(f.read(), filename, 'exec'))

rdf = False
if rdf:
    LinkExtractorRDFParser(auth = {'username': LIDO_USERNAME, 'password':LIDO_PASSWD})
else:
    parser = LinkExtractorXMLParser(auth = {'username': LIDO_USERNAME, 'password':LIDO_PASSWD})

In [106]:
fp = '/media/sf_VBox_Shared/CaseLaw/lido_rdf/2017-11-28-xml/'

failed_eclis = []

i = 0
for (ecli,) in conn.execute('select id from uitspraken_meta'):
    if i%1000 == 0:
        print(i)
    ext = '.rdf' if rdf else '.xml'
    fn = ecli.replace(':', '_') + ext
    if not os.path.exists(os.path.join(fp, fn)):
        lido_id = parser.get_lido_id(ecli)
        if rdf:
            url = "{}?id={}".format(parser.LIDO_API_URL, lido_id)
        else:
            url = "{}?id={}&output=xml".format(parser.LIDO_API_URL, lido_id)

        try:
            response = parser.get_lido_response(url)
            
            with open(os.path.join(fp, fn), 'w') as out:
                out.write(response)
        except Exception as e:
            print (ecli, e)
            failed_eclis.append(ecli)
        
    i += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
